Each file contains the history of simulation of a Markov Chain.

Assignment: Estimate the context trees through BIC.

# Packages

In [7]:
import itertools # tools for iteration
# count number of repetitions in list
from collections import Counter
import copy
import networkx as nx
import math
from pandas import DataFrame # pandas dataframe
import os # read directory

Clean directory and clone repository with data files

In [8]:
# !rm -r sample_data
# !rm -r MAE4007tasks
# !git clone --branch context-tree https://github.com/LucasMSpereira/MAE4007tasks

# Read and organize data

In [9]:
# directory with data
dataDir = './MAE4007tasks/contextTreeData'
# list of dictionaries, each referring to a file
data = []
# read each file and fill 'data'
for fileName in os.listdir(dataDir):
  # dictionary for current file
  currentData = {}
  data.append(currentData)
  # file name
  data[-1]["fileName"] = fileName
  # open, read and close file
  with open(dataDir + '/' + fileName, encoding="utf-8") as f:
    fileData = list(map(int, f.read()))
  # file data as list of integers
  data[-1]["content"] = fileData
  # size of sample
  data[-1]["size"] = len(data[-1]["content"])
  # count occurrences of each value in current file
  data[-1]["valueCount"] = dict(Counter(data[-1]["content"]))
  # unique values in current file
  data[-1]["uniqueValues"] = list(data[-1]["valueCount"].keys())
  data[-1]["uniqueValues"].sort()
for d in data:
  for f in d:
    if f != "content":
      print(f)
      print(d[f])
  print()

fileName
52845.txt
size
30000
valueCount
{1: 13668, 3: 8441, 2: 7891}
uniqueValues
[1, 2, 3]

fileName
OrderEst_ex1.txt
size
10000
valueCount
{1: 4726, 2: 5274}
uniqueValues
[1, 2]

fileName
OrderEst_ex2.txt
size
10000
valueCount
{2: 6327, 1: 3673}
uniqueValues
[1, 2]

fileName
OrderEst_ex3.txt
size
10000
valueCount
{1: 3943, 2: 2989, 3: 3068}
uniqueValues
[1, 2, 3]



# BIC for context trees

1.   Begin with the full context tree
2.   Starting from leaves, define the V value of nodes. In the case of leaves, use the likelihood of its word. In nodes, use the greater value between the likelihood of its word, and the product of the V values of its immediate successors
3.   Starting from leaves, define the binary $\chi$ value of nodes. Assign 1 if: the word's length is less than the maximum (tree depth); and the product of the V values of its immediate successors is greater than the likelihood of the word of the node. Assign 0 otherwise.
4.   Remove nodes (and respective branches) that are dependent on a node whose $\chi = 0$.

\*Assignment limits chain orders to 8. Therefore, the starting point is a tree with depth 8.

Utilities

In [10]:
def addNodeToTree(nodeID: int, word: tuple, _contextTree: nx.DiGraph, df: DataFrame, fileID: int) -> None:
  _contextTree.add_node(nodeID)
  # connect node to neighbors.
  if len(word) > 1: # if word of size at least 2
    _contextTree.add_edge( # connect to predecessor
      IDofWord(word[:len(word) - 1], df), nodeID
    )
    # connect to sucessors
    for state in data[fileID]["uniqueValues"]:
      _contextTree.add_edge(
        nodeID, IDofWord(word + (state,), df)
      )
  else: # if word of length 1
    for state in data[fileID]["uniqueValues"]: # connect to sucessors
      _contextTree.add_edge(
        nodeID, IDofWord(word + (state,), df)
      )

def calcKhi(word: tuple, fileID: int, _df: DataFrame, _tree: nx.DiGraph):
  """Determine Khi value for current node in context tree"""
  # likelihood of word
  wl = wordLikelihood(word, fileID, _df)
  # product of Vs of successors
  Vprod = sucessorsVprod(_tree, word, _df)
  if len(word) < 8 and Vprod > wl:
    return 1
  else:
    return 0

def calcV(word: tuple, _tree: nx.DiGraph, _df: DataFrame, fileID: int):
  """Determine V value for a node in the context tree"""
  wl = wordLikelihood(word, fileID, _df)
  # if node of word is a leaf
  if isLeaf(_tree, IDofWord(word, _df)):
    return wl
  else:
    return max(wl, sucessorsVprod(_tree, word, _df))

def IDofWord(word: tuple, _occurDataFrame: DataFrame):
  """ID of node of a word (same as row in dataframe)"""
  return _occurDataFrame[_occurDataFrame.word == word].index[0]

def isLeaf(tree: nx.DiGraph, nodeID: int):
  """Boolean indicating if node is a leaf"""
  return len(list(tree.successors(nodeID))) == 0

def sucessorsVprod(tree: nx.DiGraph, _word: tuple, df: DataFrame):
  """Product of V values of a node's immediate sucessors"""
  successors = tree.successors(IDofWord(_word, df))
  for node in successors:
    if "V" not in tree.nodes[node].keys():
      tree.nodes[node]["V"] = 1.0
  return math.prod([tree.nodes[node]["V"] for node in successors])
  
def wordCount(occurDataFrame: DataFrame, word: tuple) -> int:
  """Number of occurrences of certain word"""
  return occurDataFrame.at[
    occurDataFrame[occurDataFrame.word == word].index[0],
    "occurrences"
  ]

def wordLikelihood(_word: tuple, fileIndex: int, df: DataFrame):
  """Calculate likelihood of word"""
  likelihood = 1.0
  nCounts = wordCount(df, _word)
  # iterate in alphabet
  for symbol in data[fileIndex]["uniqueValues"]:
    newWordCount = wordCount(df, _word + (symbol,))
    likelihood *= (newWordCount / nCounts) ** newWordCount
  return likelihood

In [11]:
g = nx.DiGraph(nx.balanced_tree(2, 4))
list(g.successors(2))

[0, 5, 6]

Initializations

In [12]:
for f in range(len(data)):
    # directed graph for context tree
    contextTree = nx.DiGraph()
    treeOrder = 3
    # all possible words with size up to treeOrder + 1
    allWords = [
        list(itertools.product(data[f]["uniqueValues"], repeat = order))
        for order in range(1, treeOrder + 2)
    ]
    # dictionary mapping each word to number of occurrences
    occurrenceCount = [
        {"word": word, "occurrences": 0}
        for word in itertools.chain.from_iterable(allWords)
    ]
    # organize in pandas dataframe for convenience
    occurrenceCount = DataFrame(occurrenceCount)
    def studyWord(firstWordIndex: int, lastWordIndex: int, fixedIndex: int, _fileID: int, _sample: list) -> None:
        """Recursive function to count occurrences of words, and build context tree graph"""
        # word being analized
        currentWord = tuple(_sample[firstWordIndex : lastWordIndex])
        print(len(currentWord), currentWord)
        # row of current word in dataframe
        wordRow = IDofWord(currentWord, occurrenceCount)
        # increment number of times this word has been seen
        occurrenceCount.at[wordRow, "occurrences"] += 1
        # if word was found for the first time,
        # include node in graph. use dataframe word row as node ID
        if occurrenceCount.at[wordRow, "occurrences"] == 1:
            addNodeToTree(wordRow, currentWord, contextTree, occurrenceCount, _fileID)
        # update node attributes
        nx.set_node_attributes(contextTree,
            {wordRow:
            {
                "word": currentWord, "occurrences": occurrenceCount.at[wordRow, "occurrences"],
                "khi": calcKhi(currentWord, _fileID, occurrenceCount, contextTree),
                "V": calcV(currentWord, contextTree, occurrenceCount, _fileID),
                "likelihood": wordLikelihood(currentWord, _fileID, occurrenceCount)
            }
            }
        )
        # if word of size at least 2, call function on currentWord without its last symbol
        if len(currentWord) > 1:
            studyWord(firstWordIndex, lastWordIndex - 1, fixedIndex, _fileID, _sample)
        else: # if symbol
            if len(currentWord) == 1:
                return None
            studyWord(firstWordIndex + 1, fixedIndex, fixedIndex, _fileID, _sample)

    # loop in files
    sample = data[f]["content"]

    print("Counting occurrences of words of size 'order + 1'")
    # scan sample with window of size 8 to
    # count occurrences and add the words to the tree
    for i in range(len(sample) - (treeOrder) + 1 + 1):
        if i % 1000 == 0:
            print(i, "/", len(range(len(sample) - (treeOrder) + 1 + 1)))
        word = tuple(sample[i : i + (treeOrder)])
        wordRow = IDofWord(word, occurrenceCount)
        occurrenceCount.at[wordRow, "occurrences"] += 1
        addNodeToTree(wordRow, word, contextTree, occurrenceCount, f)
        nx.set_node_attributes(contextTree, {wordRow: {"V": wordLikelihood(word, f, occurrenceCount)}})

    print("Building tree")
    # scan sample with window of size 7 and apply recursive function
    # to count word occurrences and develop tree sctructure
    for i in range(len(sample) - treeOrder + 1):
        studyWord(i, i + (treeOrder - 1), i + (treeOrder - 1), f, sample)
    break

Counting occurrences of words of size 'order + 1'
0 / 29999
1000 / 29999
2000 / 29999
3000 / 29999
4000 / 29999
5000 / 29999
6000 / 29999
7000 / 29999
8000 / 29999
9000 / 29999
10000 / 29999
11000 / 29999
12000 / 29999
13000 / 29999
14000 / 29999
15000 / 29999
16000 / 29999
17000 / 29999
18000 / 29999
19000 / 29999
20000 / 29999
21000 / 29999
22000 / 29999
23000 / 29999
24000 / 29999
25000 / 29999
26000 / 29999
27000 / 29999
28000 / 29999
29000 / 29999


C:\Users\kaoid\AppData\Local\Temp\ipykernel_4924\4065165286.py:69: RuntimeWarning: overflow encountered in scalar power
  likelihood *= (newWordCount / nCounts) ** newWordCount


Building tree
2 (1, 3)
1 (1,)
2 (3, 3)
1 (3,)
2 (3, 3)
1 (3,)
2 (3, 3)
1 (3,)
2 (3, 1)
1 (3,)
2 (1, 3)
1 (1,)
2 (3, 2)
1 (3,)
2 (2, 3)
1 (2,)
2 (3, 3)
1 (3,)
2 (3, 2)
1 (3,)
2 (2, 1)
1 (2,)
2 (1, 2)
1 (1,)
2 (2, 1)
1 (2,)
2 (1, 1)
1 (1,)
2 (1, 1)
1 (1,)
2 (1, 2)
1 (1,)
2 (2, 1)
1 (2,)
2 (1, 1)
1 (1,)
2 (1, 3)
1 (1,)
2 (3, 1)
1 (3,)
2 (1, 3)
1 (1,)
2 (3, 1)
1 (3,)
2 (1, 3)
1 (1,)
2 (3, 3)
1 (3,)
2 (3, 3)
1 (3,)
2 (3, 2)
1 (3,)
2 (2, 2)
1 (2,)
2 (2, 1)
1 (2,)
2 (1, 2)
1 (1,)
2 (2, 1)
1 (2,)
2 (1, 1)
1 (1,)
2 (1, 1)
1 (1,)
2 (1, 2)
1 (1,)
2 (2, 1)
1 (2,)
2 (1, 2)
1 (1,)
2 (2, 1)
1 (2,)
2 (1, 2)
1 (1,)
2 (2, 2)
1 (2,)
2 (2, 3)
1 (2,)
2 (3, 3)
1 (3,)
2 (3, 3)
1 (3,)
2 (3, 1)
1 (3,)
2 (1, 1)
1 (1,)
2 (1, 2)
1 (1,)
2 (2, 2)
1 (2,)
2 (2, 1)
1 (2,)
2 (1, 1)
1 (1,)
2 (1, 3)
1 (1,)
2 (3, 3)
1 (3,)
2 (3, 1)
1 (3,)
2 (1, 2)
1 (1,)
2 (2, 2)
1 (2,)
2 (2, 1)
1 (2,)
2 (1, 3)
1 (1,)
2 (3, 1)
1 (3,)
2 (1, 1)
1 (1,)
2 (1, 1)
1 (1,)
2 (1, 2)
1 (1,)
2 (2, 3)
1 (2,)
2 (3, 3)
1 (3,)
2 (3, 3)
1 (3,)
2 (3, 3)
1

C:\Users\kaoid\AppData\Local\Temp\ipykernel_4924\4065165286.py:69: RuntimeWarning: overflow encountered in scalar multiply
  likelihood *= (newWordCount / nCounts) ** newWordCount


1 (2,)
2 (1, 3)
1 (1,)
2 (3, 1)
1 (3,)
2 (1, 2)
1 (1,)
2 (2, 1)
1 (2,)
2 (1, 3)
1 (1,)
2 (3, 1)
1 (3,)
2 (1, 3)
1 (1,)
2 (3, 2)
1 (3,)
2 (2, 2)
1 (2,)
2 (2, 1)
1 (2,)
2 (1, 1)
1 (1,)
2 (1, 2)
1 (1,)
2 (2, 1)
1 (2,)
2 (1, 3)
1 (1,)
2 (3, 1)
1 (3,)
2 (1, 1)
1 (1,)
2 (1, 3)
1 (1,)
2 (3, 3)
1 (3,)
2 (3, 1)
1 (3,)
2 (1, 3)
1 (1,)
2 (3, 3)
1 (3,)
2 (3, 1)
1 (3,)
2 (1, 2)
1 (1,)
2 (2, 1)
1 (2,)
2 (1, 1)
1 (1,)
2 (1, 1)
1 (1,)
2 (1, 3)
1 (1,)
2 (3, 3)
1 (3,)
2 (3, 2)
1 (3,)
2 (2, 1)
1 (2,)
2 (1, 1)
1 (1,)
2 (1, 2)
1 (1,)
2 (2, 3)
1 (2,)
2 (3, 3)
1 (3,)
2 (3, 3)
1 (3,)
2 (3, 1)
1 (3,)
2 (1, 1)
1 (1,)
2 (1, 1)
1 (1,)
2 (1, 2)
1 (1,)
2 (2, 1)
1 (2,)
2 (1, 2)
1 (1,)
2 (2, 1)
1 (2,)
2 (1, 2)
1 (1,)
2 (2, 2)
1 (2,)
2 (2, 1)
1 (2,)
2 (1, 1)
1 (1,)
2 (1, 3)
1 (1,)
2 (3, 3)
1 (3,)
2 (3, 1)
1 (3,)
2 (1, 3)
1 (1,)
2 (3, 3)
1 (3,)
2 (3, 1)
1 (3,)
2 (1, 1)
1 (1,)
2 (1, 2)
1 (1,)
2 (2, 2)
1 (2,)
2 (2, 3)
1 (2,)
2 (3, 2)
1 (3,)
2 (2, 1)
1 (2,)
2 (1, 3)
1 (1,)
2 (3, 3)
1 (3,)
2 (3, 3)
1 (3,)
2 (3, 1)
1 (3,)
2